In [ ]:
## cloning repo
#!git clone https://{secret_value_0}@github.com/wildsonbbl/gnnepcsaft.git --branch san --single-branch -q
## from within repo
%cd ../

In [ ]:
import os.path as osp, os

os.environ["CUDA_VISIBLE_DEVICES"] = ""
import torch, numpy as np, polars as pl
from gnnepcsaft.data.graphdataset import ThermoMLDataset, Ramirez
from gnnepcsaft.train.train import create_model
from gnnepcsaft.train.utils import calc_deg
from gnnepcsaft.train.utils import mape
from gnnepcsaft.demo.utils import model_para_fn

torch.cuda.is_available()
ra_loader = Ramirez("gnnepcsaft/data/ramirez2022")
ra_para = {}
for graph in ra_loader:
    inchi, para = graph.InChI, graph.para.view(-1, 3).round(decimals=2)
    ra_para[inchi] = para.tolist()[0]
path = osp.join("gnnepcsaft/data", "thermoml")
testloader = ThermoMLDataset(path)
device = torch.device("cpu")
from configs.default import get_config

config = get_config()

In [ ]:
config.model = "PNA"
config.propagation_depth = 2
config.hidden_dim = 128
config.num_mlp_layers = 1
config.pre_layers = 1
config.post_layers = 3
model_dtype = torch.float64

In [ ]:
# Create and initialize the network.
deg = calc_deg("ramirez", "gnnepcsaft/")
model = create_model(config, deg).to(device, model_dtype)

In [ ]:
# Set up checkpointing of the model.
ckp_path = "gnnepcsaft/train/checkpoints/model5-8_25e6.pth"
if osp.exists(ckp_path):
    checkpoint = torch.load(ckp_path, map_location=torch.device("cpu"))
    model.load_state_dict(checkpoint["model_state_dict"])
    print(f"model checkpoint step {checkpoint['step']}")
    del checkpoint

In [ ]:
model_para, model_array = model_para_fn(model)

In [ ]:
para3 = {}
for inchi in model_para:
    if inchi not in ra_para:
        para3[inchi] = model_para[inchi]

In [ ]:
len(ra_para), len(model_para), len(para3)

In [ ]:
import pickle

with open("gnnepcsaft/data/thermoml/processed/para3.pkl", "wb") as file:
    # A new file will be created
    pickle.dump(para3, file)

In [ ]:
import pickle

with open("gnnepcsaft/data/thermoml/raw/para3_fitted.pkl", "wb") as file:
    # A new file will be created
    pickle.dump(para3, file)